In [4]:
%pip install streamlit

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 44.3/44.3 kB 2.1 MB/s eta 0:00:00
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached smmap-5.0.1-py3-none-any.wh

In [2]:
%pip install scikit-learn

  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import mysql.connector

In [2]:
import mysql.connector
import pandas as pd

In [5]:
# Conexión a la base de datos
conexion = mysql.connector.connect(
    host="127.0.0.1",
    user="root",  
    password="11jablum11",
    database="yfinance"  
)

# Carga de datos desde una consulta SQL a un DataFrame
query = "SELECT precios_historicos.precio_apertura,precios_historicos.precio_cierre,precios_historicos.maximo,precios_historicos.minimo,precios_historicos.volumen,empresas_sp500.sector,empresas_sp500.industria FROM precios_historicos INNER JOIN empresas_sp500 ON precios_historicos.id_empresa = empresas_sp500.id_empresa;"
df = pd.read_sql(query, conexion)





/var/folders/3l/2mfq68kd2m7_kpj_3qkpl1qw0000gn/T/ipykernel_85637/3032743289.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conexion)


KeyboardInterrupt: 

In [7]:
conexion = mysql.connector.connect(
    host="127.0.0.1",
    user="root",  
    password="11jablum11",
    database="yfinance")

query = "SELECT precios_historicos.precio_apertura,precios_historicos.precio_cierre,precios_historicos.maximo,precios_historicos.minimo,precios_historicos.volumen,empresas_sp500.sector,empresas_sp500.industria FROM precios_historicos INNER JOIN empresas_sp500 ON precios_historicos.id_empresa = empresas_sp500.id_empresa;"
empresas_df = pd.read_sql(query, conexion)


empresas_df.head(5)

/var/folders/3l/2mfq68kd2m7_kpj_3qkpl1qw0000gn/T/ipykernel_85637/2270608046.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empresas_df = pd.read_sql(query, conexion)


,precio_apertura,precio_cierre,maximo,minimo,volumen,sector,industria
0,20.1681,19.8138,20.2600,19.7482,2599386,Industrials,Industrial Conglomerates
1,19.4989,19.0265,19.9057,19.0265,3245705,Industrials,Industrial Conglomerates
2,19.1315,19.5776,20.2075,19.1315,4424482,Industrials,Industrial Conglomerates
3,19.8007,21.1523,21.5197,19.8007,7147057,Industrials,Industrial Conglomerates
4,21.2310,21.5721,21.7952,20.9817,4905035,Industrials,Industrial Conglomerates


In [8]:
import numpy as np

In [9]:
empresas_df['cluster'] = np.nan

In [12]:
# Simular datos de ejemplo
np.random.seed(42)

empresas_df = pd.DataFrame(empresas_df, columns=['precio_apertura', 'precio_cierre', 'maximo', 'minimo', 'volumen'])
x = empresas_df[['precio_apertura','precio_cierre','maximo','minimo','volumen']]


# Ejecutar K-Means para agrupar las acciones
kmeans = KMeans(n_clusters=5, random_state=42)
y = kmeans.fit_predict(x)
empresas_df['Cluster'] = y


print(empresas_df.head())  # Visualizar las características y el cluster asignado

conexion.close()

   precio_apertura  precio_cierre   maximo   minimo  volumen  Cluster
0          20.1681        19.8138  20.2600  19.7482  2599386        3
1          19.4989        19.0265  19.9057  19.0265  3245705        3
2          19.1315        19.5776  20.2075  19.1315  4424482        3
3          19.8007        21.1523  21.5197  19.8007  7147057        3
4          21.2310        21.5721  21.7952  20.9817  4905035        3


In [13]:
#convertir Y en unidimensional#
y = empresas_df['Cluster'].values.ravel()

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print("Datos preparados:")
print(f"Tamaño de X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"Tamaño de y_train: {y_train.shape}, y_test: {y_test.shape}")

Datos preparados:
Tamaño de X_train: (2205567, 5), X_test: (551392, 5)
Tamaño de y_train: (2205567,), y_test: (551392,)


In [14]:
print(y.shape)

(2756959,)


In [15]:
import pickle

In [16]:
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Crear y entrenar el modelo
modelo = RandomForestClassifier(n_estimators=20,max_depth=10,max_features= 'sqrt', random_state=42)
modelo.fit(X_train, y_train) #Tuve que reducir significativamente los parámetros por motivo de la RAM 

# Hacer predicciones en los datos de prueba
pca = PCA(n_components=5)  # Reduce a 5 componentes principales
X_test_reduced = pca.fit_transform(X_test)
y_pred = modelo.predict(X_test_reduced)

print("Dimensiones de y_test:", y_test.shape)
print("Dimensiones de y_pred:", y_pred.shape)

# Convertir y_test a un array de NumPy
y_test = np.array(y_test).ravel()
y_pred = np.array(y_pred).ravel()

with open('modelo_entrenado.pkl','wb') as archivo_pickle:
    pickle.dump(modelo,archivo_pickle)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Dimensiones de y_test: (551392,)
Dimensiones de y_pred: (551392,)
Matriz de Confusión:
[[     0      0      0   9357      0]
 [     0      0      0    575      0]
 [     0      0      0   1593      0]
 [     0      0      0 539784      0]
 [     0      0      0     83      0]]

Reporte de Clasificación:


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      9357
           1       0.00      0.00      0.00       575
           2       0.00      0.00      0.00      1593
           3       0.98      1.00      0.99    539784
           4       0.00      0.00      0.00        83

    accuracy                           0.98    551392
   macro avg       0.20      0.20      0.20    551392
weighted avg       0.96      0.98      0.97    551392



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
import mysql.connector

In [7]:


# Configuración de la base de datos
st.sidebar.subheader("Configuración de Base de Datos")
db_host = st.sidebar.text_input("Host de la Base de Datos", value="localhost")
db_user = st.sidebar.text_input("Usuario", value="root")
db_password = st.sidebar.text_input("Contraseña", type="password")
db_name = st.sidebar.text_input("Nombre de la Base de Datos", value="Yfinance")

# Conectar y cargar datos
if st.sidebar.button("Conectar y Cargar Datos"):
    try:
        # Conectar a la base de datos
        conexion = mysql.connector.connect(
            host=db_host,
            user=db_user,
            password=db_password,
            database=db_name
        )
        st.success("Conexión exitosa a la base de datos")

        # Consulta SQL
        query = "SELECT precios_historicos.precio_apertura,precios_historicos.precio_cierre,precios_historicos.maximo,precios_historicos.minimo,precios_historicos.volumen,empresas_sp500.sector,empresas_sp500.industria FROM precios_historicos INNER JOIN empresas_sp500 ON precios_historicos.id_empresa = empresas_sp500.id_empresa;"
        empresas_df = pd.read_sql(query, conexion)

        st.write("Datos cargados desde la base de datos:")
        st.dataframe(empresas_df.head())

        # Clustering con K-Means
        columnas_numericas = ['precio_apertura', 'precio_cierre', 'maximo', 'minimo', 'volumen']
        X = empresas_df[columnas_numericas].dropna()

        st.sidebar.subheader("Configuración de K-Means")
        n_clusters = st.sidebar.slider("Número de Clusters (K)", min_value=2, max_value=10, value=5, step=1)

        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        clusters = kmeans.fit_predict(X)

        empresas_df['Cluster'] = clusters

        st.write("Clustering completado. Vista previa de los clusters:")
        st.dataframe(empresas_df[['Cluster'] + columnas_numericas].head())

        # Clasificación con RandomForest
        st.sidebar.subheader("Entrenar Modelo de Clasificación")
        max_depth = st.sidebar.slider("Profundidad Máxima del Árbol", min_value=2, max_value=20, value=10, step=1)
        n_estimators = st.sidebar.slider("Número de Estimadores", min_value=10, max_value=100, value=20, step=10)

        y = empresas_df['Cluster']
        X_clasificacion = X

        modelo = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        modelo.fit(X_clasificacion, y)

        predicciones = modelo.predict(X_clasificacion)

        st.write("Resultados del modelo de clasificación:")
        empresas_df['Predicción_Cluster'] = predicciones
        st.dataframe(empresas_df[['Cluster', 'Predicción_Cluster'] + columnas_numericas].head())

        # Cerrar la conexión
        conexion.close()
    except Exception as e:
        st.error(f"Error al conectar con la base de datos: {e}")

# Nota final
st.sidebar.info("Desarrollado con Streamlit")


2024-12-02 10:43:59.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 10:43:59.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 10:43:59.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 10:43:59.022 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 10:43:59.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 10:43:59.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 10:43:59.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 10:43:59.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())